# Perfilamiento y preprocesamiento de los datos

En este notebook se realizará el perfilamiento y preprocesamiento de los datos suministrados de la base de datos de WWI.

## Carga de librerías

In [1]:
import pandas as pd
import numpy as np

## Dimensión _City_

Carga del archivo:

In [2]:
df_city = pd.read_csv('dimension_city.csv', sep=',', encoding = 'utf-8')
df_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   row ID                      98 non-null     object
 1   City_Key                    98 non-null     int64 
 2   City                        98 non-null     object
 3   State_Province              97 non-null     object
 4   Country                     97 non-null     object
 5   Continent                   97 non-null     object
 6   Sales_Territory             97 non-null     object
 7   Region                      97 non-null     object
 8   Subregion                   97 non-null     object
 9   Latest_Recorded_Population  98 non-null     int64 
dtypes: int64(2), object(8)
memory usage: 7.8+ KB


In [3]:
df_city.head()

,row ID,City_Key,City,State_Province,Country,Continent,Sales_Territory,Region,Subregion,Latest_Recorded_Population
0,Row0,0,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Row1,1,Carrollton,New York,United States,North America,Mideast,Americas,Northern America,0
2,Row2,2,Carrollton,Virginia,United States,North America,Southeast,Americas,Northern America,4574
3,Row3,3,Carrollton,Illinois,United States,North America,Great Lakes,Americas,Northern America,2484
4,Row4,4,Carrollton,Missouri,United States,North America,Plains,Americas,Northern America,3784


In [4]:
df_city.isnull().sum()

row ID                        0
City_Key                      0
City                          0
State_Province                1
Country                       1
Continent                     1
Sales_Territory               1
Region                        1
Subregion                     1
Latest_Recorded_Population    0
dtype: int64

Se puede ver que existen pocos registros con valores nulos. También hay una columna que es innecesaria. 

Entonces, se elimina la primera fila, que es la que contiene los nulos y además no tiene ninguna información. También se elimina la columna `rowID` ya que aporta la misma infromación que la columna `City_key`

In [5]:
df_city_clean = df_city.dropna()
df_city_clean = df_city_clean.drop(["row ID"], axis=1)
df_city_clean.head()

,City_Key,City,State_Province,Country,Continent,Sales_Territory,Region,Subregion,Latest_Recorded_Population
1,1,Carrollton,New York,United States,North America,Mideast,Americas,Northern America,0
2,2,Carrollton,Virginia,United States,North America,Southeast,Americas,Northern America,4574
3,3,Carrollton,Illinois,United States,North America,Great Lakes,Americas,Northern America,2484
4,4,Carrollton,Missouri,United States,North America,Plains,Americas,Northern America,3784
5,5,Carrollton,Ohio,United States,North America,Great Lakes,Americas,Northern America,3241


Ahora, se observa que hay algunos de registros de ciudades con "Latest_Recorded_Population" en 0. Esto no tiene sentido con el mundo real, por lo que se cambiará este valor por el promedio de población de las demás ciudades.

In [6]:
(sum,count) = 0,0
for i in df_city_clean["Latest_Recorded_Population"]:
    sum += i
    if i != 0: count += 1
avg = int(sum/count)
print("El promedio es :", avg)

El promedio es : 30420


In [7]:
df_city_clean["Latest_Recorded_Population"] = df_city_clean.Latest_Recorded_Population.replace({0:avg})
df_city_clean.head()

,City_Key,City,State_Province,Country,Continent,Sales_Territory,Region,Subregion,Latest_Recorded_Population
1,1,Carrollton,New York,United States,North America,Mideast,Americas,Northern America,30420
2,2,Carrollton,Virginia,United States,North America,Southeast,Americas,Northern America,4574
3,3,Carrollton,Illinois,United States,North America,Great Lakes,Americas,Northern America,2484
4,4,Carrollton,Missouri,United States,North America,Plains,Americas,Northern America,3784
5,5,Carrollton,Ohio,United States,North America,Great Lakes,Americas,Northern America,3241


In [8]:
df_city_clean["Sales_Territory"].unique()

array(['Mideast', 'Southeast', 'Great Lakes', 'Plains', 'Southwest',
       'Far West', 'Rocky Mountain', 'External'], dtype=object)

Finalemente, se guarda el archivo con las modificaciones para seguir con el proceso ETL.

In [9]:
df_city_clean.to_csv("Muestra_limpios/dimension_city.csv", encoding='utf-8', index=False)

## Dimensión _Customer_

Carga del archivo:

In [10]:
df_customer = pd.read_csv('dimension_customer.csv', sep=',')
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Customer_Key      403 non-null    int64  
 1   Customer          403 non-null    object 
 2   Bill_To_Customer  402 non-null    object 
 3   Category          402 non-null    object 
 4   Buying_Group      402 non-null    object 
 5   Primary_Contact   402 non-null    object 
 6   Postal_Code       402 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 22.2+ KB


In [11]:
df_customer.head()

,Customer_Key,Customer,Bill_To_Customer,Category,Buying_Group,Primary_Contact,Postal_Code
0,0,Unknown,NaN,NaN,NaN,NaN,NaN
1,1,Tailspin Toys (Head Office),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Waldemar Fisar,90410.0
2,2,Tailspin Toys (Sylvanite- MT),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Lorena Cindric,90216.0
3,3,Tailspin Toys (Peeples Valley- AZ),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Bhaargav Rambhatla,90205.0
4,4,Tailspin Toys (Medicine Lodge- KS),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Daniel Roman,90152.0


In [12]:
df_customer.isnull().sum()

Customer_Key        0
Customer            0
Bill_To_Customer    1
Category            1
Buying_Group        1
Primary_Contact     1
Postal_Code         1
dtype: int64

Se puede ver que existe un registro en algunas columnas reconocidos como nulos. Estos corresponden a la primera fila del archivo. También, podemos notar que toma los valores de la columna `Postal_code` como `float`. Arreglaremos estos problemas eliminando la primera fila y convirtiendo los valores a enteros, respectivamente.

In [13]:
df_customer_clean = df_customer.dropna()
df_customer_clean = df_customer_clean.astype({"Postal_Code":int})
df_customer_clean.head()

,Customer_Key,Customer,Bill_To_Customer,Category,Buying_Group,Primary_Contact,Postal_Code
1,1,Tailspin Toys (Head Office),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Waldemar Fisar,90410
2,2,Tailspin Toys (Sylvanite- MT),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Lorena Cindric,90216
3,3,Tailspin Toys (Peeples Valley- AZ),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Bhaargav Rambhatla,90205
4,4,Tailspin Toys (Medicine Lodge- KS),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Daniel Roman,90152
5,5,Tailspin Toys (Gasport- NY),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Johanna Huiting,90261


Finalemente, se guarda el archivo con las modificaciones para seguir con el proceso ETL.

In [14]:
df_customer_clean.to_csv("Muestra_limpios/dimension_customer.csv", encoding='utf-8', index=False)

## Dimensión _Date_

Carga del archivo:

In [15]:
df_date = pd.read_csv('dimension_date.csv', sep=',', encoding = 'utf-8')
df_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Date_key               1461 non-null   object
 1   Day_Number             1461 non-null   int64 
 2   Day_val                1461 non-null   int64 
 3   Month_val              1461 non-null   object
 4   Short_Month            1461 non-null   object
 5   Calendar_Month_Number  1461 non-null   int64 
 6   Calendar_Year          1461 non-null   int64 
 7   Fiscal_Month_Number    1461 non-null   int64 
 8   Fiscal_Year            1461 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 102.9+ KB


In [16]:
df_date.head()

,Date_key,Day_Number,Day_val,Month_val,Short_Month,Calendar_Month_Number,Calendar_Year,Fiscal_Month_Number,Fiscal_Year
0,2013-01-01,1,1,January,Jan,1,2013,3,2013
1,2013-01-02,2,2,January,Jan,1,2013,3,2013
2,2013-01-03,3,3,January,Jan,1,2013,3,2013
3,2013-01-04,4,4,January,Jan,1,2013,3,2013
4,2013-01-05,5,5,January,Jan,1,2013,3,2013


Este archivo no parece tener problemas, entonces se deja igual como está

In [17]:
df_date.to_csv("Muestra_limpios/dimension_date.csv", encoding='utf-8', index=False)

## Dimensión _Employee_

Carga del archivo:

In [18]:
df_employee = pd.read_csv('dimension_employee.csv', sep=',', encoding = 'utf-8')
df_employee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Employee_Key    213 non-null    int64 
 1   Employee        213 non-null    object
 2   Preferred_Name  212 non-null    object
 3   Is_Salesperson  213 non-null    bool  
dtypes: bool(1), int64(1), object(2)
memory usage: 5.3+ KB


In [19]:
df_employee.head()

,Employee_Key,Employee,Preferred_Name,Is_Salesperson
0,0,Unknown,NaN,False
1,1,Lily Code,Lily,True
2,2,Isabella Rupp,Isabella,False
3,3,Ethan Onslow,Ethan,False
4,4,Amy Trefl,Amy,True


El único problema con este archivo parece ser el primer registro, que contiene valores nulos, de una forma similar como en casos anteriores. Entonces, basta solo con borrar esta fila para arreglar el archivo

In [20]:
df_employee_clean = df_employee.dropna()
df_employee_clean.head()

,Employee_Key,Employee,Preferred_Name,Is_Salesperson
1,1,Lily Code,Lily,True
2,2,Isabella Rupp,Isabella,False
3,3,Ethan Onslow,Ethan,False
4,4,Amy Trefl,Amy,True
5,5,Jai Shand,Jai,False


Finalemente, se guarda el archivo con las modificaciones para seguir con el proceso ETL.

In [21]:
df_employee_clean.to_csv("Muestra_limpios/dimension_employee.csv", encoding='utf-8', index=False)

## Dimensión _Stock Item_

Carga del archivo:

In [22]:
df_stock = pd.read_csv('dimension_stock_item.csv', sep=',')
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Stock_Item_Key            672 non-null    int64 
 1   Stock_Item                672 non-null    object
 2   Color                     360 non-null    object
 3   Selling_Package           671 non-null    object
 4   Buying_Package            671 non-null    object
 5   Brand                     67 non-null     object
 6   Size_val                  468 non-null    object
 7   Lead_Time_Days            672 non-null    int64 
 8   Quantity_Per_Outer        672 non-null    int64 
 9   Is_Chiller_Stock          672 non-null    bool  
 10  Tax_Rate                  672 non-null    object
 11  Unit_Price                672 non-null    object
 12  Recommended_Retail_Price  672 non-null    object
 13  Typical_Weight_Per_Unit   672 non-null    object
dtypes: bool(1), int64(3), obje

In [23]:
df_stock.head()

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Brand,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
0,0,Unknown,NaN,NaN,NaN,NaN,NaN,0,0,False,",000",",00",",00",",000"
1,1,Void fill 400 L bag (White) 400L,NaN,Each,Each,NaN,400L,14,10,False,"14,000","50,00","74,75","1,000"
2,2,Void fill 300 L bag (White) 300L,NaN,Each,Each,NaN,300L,14,10,False,"14,000","37,50","56,06",",750"
3,3,Void fill 200 L bag (White) 200L,NaN,Each,Each,NaN,200L,14,10,False,"14,000","25,00","37,38",",500"
4,4,Void fill 100 L bag (White) 100L,NaN,Each,Each,NaN,100L,14,10,False,"14,000","12,50","18,69",",250"


En primer lugar, es posible ver que tiene el mismo problema de la primera fila que han tenido algunos de los archivos anteriores. Luego, vemos que hay una gran cantidad de registros "nulos" en varias columnas, pero como no tenemos un diccionario de datos o información sobre los posibles valores de las columnas vamos a suponer que estos casos no son un error sino una posibilidad en la realidad del negocio. Finalmente, vemos que las últimas 4 columnas no logran capturar bien la información, pues son almacenadas como `object` y todas corresponden a valores `float`. Trataremos cada caso a continuación.

Primero, empezamos eliminando la primera fila

In [24]:
df_stock_clean = df_stock.drop(0)
df_stock_clean.head(3)

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Brand,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
1,1,Void fill 400 L bag (White) 400L,NaN,Each,Each,NaN,400L,14,10,False,"14,000","50,00","74,75","1,000"
2,2,Void fill 300 L bag (White) 300L,NaN,Each,Each,NaN,300L,14,10,False,"14,000","37,50","56,06",",750"
3,3,Void fill 200 L bag (White) 200L,NaN,Each,Each,NaN,200L,14,10,False,"14,000","25,00","37,38",",500"


Ahora, se arreglarán los valores de las últimas columnas que no están siendo leídos correctamente

In [25]:
# Se crea una función que permita arreglar el formato de los números
def to_float(x):
    if x.startswith(","):
        x = x.replace(",","0.")
    else:
        x = x.replace(",",".")
    return x

In [26]:
# Se aplica la función creada a cada columna
df_stock_clean["Tax_Rate"] = df_stock_clean["Tax_Rate"].apply(to_float)
df_stock_clean["Unit_Price"] = df_stock_clean["Unit_Price"].apply(to_float)
df_stock_clean["Recommended_Retail_Price"] = df_stock_clean["Recommended_Retail_Price"].apply(to_float)
df_stock_clean["Typical_Weight_Per_Unit"] = df_stock_clean["Typical_Weight_Per_Unit"].apply(to_float)

# Se establecen los valores como float
df_stock_clean = df_stock_clean.astype({"Tax_Rate":float,"Unit_Price":float,"Recommended_Retail_Price":float,"Typical_Weight_Per_Unit":float})

df_stock_clean.head(3)

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Brand,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
1,1,Void fill 400 L bag (White) 400L,NaN,Each,Each,NaN,400L,14,10,False,14.0,50.0,74.75,1.00
2,2,Void fill 300 L bag (White) 300L,NaN,Each,Each,NaN,300L,14,10,False,14.0,37.5,56.06,0.75
3,3,Void fill 200 L bag (White) 200L,NaN,Each,Each,NaN,200L,14,10,False,14.0,25.0,37.38,0.50


Finalemente, se guarda el archivo con las modificaciones para seguir con el proceso ETL.

In [27]:
df_stock_clean.to_csv("Muestra_limpios/dimension_stock_item.csv", encoding='utf-8', index=False)

## Tabla de hechos _Order_

Carga del archivo:

In [28]:
df_order = pd.read_csv('fact_order.csv', sep=',', encoding = 'utf-8')
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   order_key            1000 non-null   int64  
 1   city_key             1000 non-null   int64  
 2   customer_key         1000 non-null   int64  
 3   stock_item_key       1000 non-null   int64  
 4   order_date_key       1000 non-null   object 
 5   picked_date_key      1000 non-null   object 
 6   salesperson_key      1000 non-null   int64  
 7   picker_key           1000 non-null   int64  
 8   package              1000 non-null   object 
 9   quantity             1000 non-null   int64  
 10  unit_price           1000 non-null   float64
 11  tax_rate             1000 non-null   int64  
 12  total_excluding_tax  1000 non-null   float64
 13  tax_amount           1000 non-null   float64
 14  total_including_tax  1000 non-null   float64
dtypes: float64(4), int64(8), object(3)
memo

In [29]:
df_order.head()

,order_key,city_key,customer_key,stock_item_key,order_date_key,picked_date_key,salesperson_key,picker_key,package,quantity,unit_price,tax_rate,total_excluding_tax,tax_amount,total_including_tax
0,1,91,179,533,2014-02-05,2013-03-17,135,122,S,805,237.77,61,2245.39,305.40,7529.12
1,2,83,2,631,2015-11-17,2013-07-19,2,133,S,76,721.71,59,6585.70,135.12,8442.06
2,3,47,390,174,2015-04-29,2015-11-03,128,75,S,585,2866.71,9,8113.37,792.11,9934.49
3,4,8,218,157,2015-04-04,2014-12-03,84,191,S,878,4671.07,39,9109.56,668.65,6166.09
4,5,94,167,235,2015-01-26,2015-01-11,91,197,S,583,1950.68,24,9277.41,66.55,3885.52


Este archivo no parece tener problemas, entonces se deja igual como está

In [30]:
df_order.to_csv("Muestra_limpios/fact_order.csv", encoding='utf-8', index=False)